# Obtaining Learning curves for programming language in Stackoverflow

In [2]:
#Import Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Import Tables

In [4]:
RanswersTable = pd.read_csv("r_answers.csv")
RquestionsTable = pd.read_csv("r_questions.csv")
JAVAanswersTable = pd.read_csv("java_answers.csv")
JAVAquestionsTable = pd.read_csv("java_questions.csv")
#PYTanswersTable = pd.read_csv("python_answers.csv")
#PYTquestionsTable=pd.read_csv("python_questions.csv")

C:\Users\baren\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Task 1 - Compute Probabilities of answers getting accpeted for each users at several time points.

In [5]:
def addProbAcceptedAnswerToTable(answersTable_provided,questionsTable_provided):
    
    answersTable=answersTable_provided.copy()
    questionsTable=questionsTable_provided.copy()
    answersTable=answersTable.sort_values([' OwnerUserId',' CreationDate']) #order by user and then by creation date
    questionsTable=questionsTable.sort_values([' OwnerUserId',' CreationDate']) #order by user and then by creation date
    
    #Find answers that were accepted (note that this information is in questions table)
    accepted=pd.match(answersTable["Id"].tolist(),questionsTable[' AcceptedAnswerId'])
    
    #Update answersTable with infor regarding whether answer was the accepted one or not.
    answersTable["AcceptedAnswer"]=accepted
    answersTable['AcceptedAnswer'] = np.where(answersTable['AcceptedAnswer'] == -1,0, answersTable['AcceptedAnswer'])
    answersTable['AcceptedAnswer'] = np.where(answersTable['AcceptedAnswer'] != 0,1, answersTable['AcceptedAnswer'])
    
    #Compute cumulative sum of accepted answers so far.
    answersTable['CumSumAccepted'] = answersTable.groupby(' OwnerUserId')['AcceptedAnswer'].cumsum()

    #Compute number of questions answered so far.
    answersTable['CumSumQuestions'] = answersTable.groupby(' OwnerUserId').cumcount() + 1

    #Compute prob of accepted answer
    answersTable['ProbAccAnswer'] = answersTable['CumSumAccepted']/answersTable['CumSumQuestions']
    
    return(answersTable)
 

In [6]:
#Compute probabilities of answer getting accepted for R, JAVA and Python
RanswersTable = addProbAcceptedAnswerToTable(RanswersTable,RquestionsTable)
JAVAanswersTable = addProbAcceptedAnswerToTable(JAVAanswersTable,JAVAquestionsTable)


C:\Users\baren\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: pd.match() is deprecated and will be removed in a future version
  if __name__ == '__main__':


## Task2 - Identify First Activity for each User

In [ ]:
def gettingTableUserFirstActivity(answersTable,questionsTable):
    #"Normalization": 0 should correspond to time of first activity so that time is read as days after first activity
    #Correcting format of dates in the table
    answersTable[' CreationDate']=pd.to_datetime(answersTable[' CreationDate'])
    questionsTable[' CreationDate']=pd.to_datetime(questionsTable[' CreationDate'])
    
    #2 create relativeTime
    #2.1 identify the first activity of users
    answersTable["FirstActivity"]=~answersTable[" OwnerUserId"].duplicated()
    questionsTable["FirstActivity"]=~questionsTable[" OwnerUserId"].duplicated()
    
    #obtaining first answer and first question of a user
    firstAnswers=answersTable.loc[answersTable["FirstActivity"]==True][[" OwnerUserId"," CreationDate"]]
    firstQuestions = questionsTable.loc[questionsTable["FirstActivity"]==True][[" OwnerUserId"," CreationDate"]]
    
    #and merge them into one table
    firstActTable=firstQuestions.merge(firstAnswers, left_on=" OwnerUserId", right_on=" OwnerUserId", how='outer')
    firstActTable.columns = [" OwnerUserId","FirstQuestion","FirstAnswer"]
    
    #obtain first activity (either answer or question)
    firstActTable["FirstActivity"]=firstActTable[["FirstQuestion","FirstAnswer"]].min(axis=1)
    
    return(firstActTable)

In [ ]:
RfirstActTable = gettingTableUserFirstActivity(RanswersTable,RquestionsTable)
JAVAfirstActTable = gettingTableUserFirstActivity(JAVAanswersTable,JAVAquestionsTable)

In [ ]:
JAVAanswersTable[' CreationDate']=pd.to_datetime(JAVAanswersTable[' CreationDate'],errors="coerce")

In [ ]:
JAVAanswersTable[JAVAanswersTable[" CreationDate"].isnull()]

In [ ]:
JAVAquestionsTable[' CreationDate']=pd.to_datetime(JAVAquestionsTable[' CreationDate'],errors="coerce")